In [1]:
import torch

torch.__version__

'2.4.0+cu124'

# 02. Neural Classification with Pytorch
classification is a problem of predicting whether something is one thing or another

## 1. Buat klasisfikasi and get it ready

In [2]:
import sklearn
sklearn.__version__

'1.5.1'

In [3]:
from sklearn.datasets import make_circles

# make 1000 sample
n_samples = 1000

# create circle
X, y = make_circles(n_samples, noise=0.03, random_state=42)

In [6]:
len(X), len(y)

(1000, 1000)